## GA with random edge start population



In [1]:
import sys; sys.path.append('..')

In [2]:
import math
from os import path
import random

In [3]:
import networkx as nx
from matplotlib import pyplot as plt

In [4]:
from graph import Graph
from graph.disjointsets import DisjointSets
from graph.reader import read_problem
from graph.steiner import prunning_mst
from graph.util import has_cycle, how_many_components

In [5]:
from base.binary.combiner import crossover_1point, crossover_2points, crossover_uniform
from base.chromosome import random_binary
from base.condition import BestKnownReached, BestSolutionKnowReached, Stagnation
from base.customevol import SteinerEvolution as Evolution
from base.customevol import SteinerPopulation as Population
from base.mutate import flip_onebit
from base.normalization import normalize
from base.pickers import random_picker
from base.selector import roullete
from base.tracker import DataTracker
from base.util import display, update_best, update_generation

from treetools import Eval, Converter

In [6]:
from pxsimpliest import SimpliestPX

In [7]:
from draw import hierarchy_pos

---

In [8]:
# problema = path.join('..', 'datasets', 'ORLibrary', 'steinc1.txt')
problema = path.join('..', 'datasets', 'ORLibrary', 'steinb16.txt')

STPG = read_problem(problema)

In [9]:
STPG.nro_nodes

100

In [10]:
STPG.nro_edges

200

In [11]:
STPG.nro_terminals

17

In [12]:
print(STPG.terminals)

{97, 37, 69, 72, 9, 76, 60, 81, 82, 17, 20, 21, 54, 87, 25, 59, 28}


In [13]:
tamanho_populacao = 50

In [14]:
funcao_avaliacao = Eval(STPG)

In [15]:
terminals = set(STPG.terminals)

def is_terminal(vertice):
    return vertice in terminals

In [16]:
def prunning(treegraph):
    
    to_eliminate = set()
    
    for v in treegraph.vertices:
        degree = treegraph.degree(v)
        if (not is_terminal(v)) and (degree == 1):
            to_eliminate.add(v)
        elif (degree == 0) and (not is_terminal(v)) :
            treegraph.remove_node(v)
    
    checkbefore = set()
    
    while to_eliminate:
        while to_eliminate:
            v = to_eliminate.pop()
            for adj in treegraph.adjacent_to(v):
                checkbefore.add(adj)
            treegraph.remove_node(v)

        while checkbefore:
            u = checkbefore.pop()
            if (not is_terminal(u)) and (treegraph.degree(u) == 1):
                to_eliminate.add(u)
                
    return treegraph   

In [17]:
def random_treegraph_chromosome(STPG):
    
    graph = STPG.graph
    terminals = set(STPG.terminals)
    
    alledges = [(v, u, graph.weight(v,u)) for v, u in graph.gen_undirect_edges()]
    random.shuffle(alledges) # random component

    DS = DisjointSets()
    subgraph = Graph()
    total_cost = 0
    
    for v in graph.vertices:
        DS.make_set(v)

    while alledges :
        v, u, weight = alledges.pop()

        if DS.find(v) != DS.find(u):
            DS.union(v, u)
            total_cost += weight
            subgraph.add_edge(v, u, weight=weight)
            terminals.discard(v)
            terminals.discard(u)

    return prunning(subgraph)

In [18]:
gg = random_treegraph_chromosome(STPG)

In [19]:
funcao_avaliacao(gg)

(262, 1)

In [20]:
gg = prunning(gg)

In [21]:
funcao_avaliacao(gg)

(262, 1)

In [22]:
STPG.terminals - set(gg.vertices)

set()

In [23]:
gg.edges

defaultdict(dict,
            {37: {91: 5},
             91: {37: 5, 72: 8, 92: 7},
             23: {47: 6, 33: 10},
             19: {79: 7, 18: 4, 62: 5},
             79: {19: 7, 52: 1, 32: 2},
             12: {17: 10, 64: 1},
             17: {12: 10},
             81: {30: 8},
             72: {52: 7, 91: 8},
             52: {72: 7, 79: 1},
             43: {32: 2, 3: 9, 51: 2, 5: 9},
             32: {43: 2, 97: 3, 79: 2},
             64: {57: 4, 12: 1},
             30: {81: 8, 18: 5},
             21: {60: 2},
             60: {21: 2, 47: 8, 61: 8},
             18: {19: 4, 30: 5},
             2: {3: 7, 69: 10},
             3: {2: 7, 43: 9},
             85: {20: 2, 11: 8},
             28: {40: 10},
             40: {28: 10, 83: 10},
             62: {19: 5, 57: 9},
             31: {87: 6, 57: 8},
             57: {62: 9, 64: 4, 31: 8},
             47: {60: 8, 23: 6, 83: 10},
             92: {76: 9, 11: 7, 91: 7},
             76: {92: 9},
             11: {92: 7, 85:

---

In [24]:
populacao = (Population(chromosomes=[ random_treegraph_chromosome(STPG) for _ in range(tamanho_populacao) ],
                        eval_function=funcao_avaliacao,
                        maximize=True)
                        .evaluate()
                        .callback(normalize)
                        .callback(update_best))

In [25]:
len(populacao)

50

In [26]:
populacao.documented_best.cost

232

In [27]:
evolution =  (Evolution()
                .evaluate()
                .callback(normalize)
                .callback(update_best)
                .select(selection_func=roullete)
                .crossover(combiner= SimpliestPX(STPG), parent_picker=random_picker)
                .callback(update_generation)
                .callback(display, every=100))

In [28]:
# %%time 

# resultado = populacao.evolve(evolution, n=500)

In [29]:
# resultado.runtime

In [30]:
# resultado.documented_best.cost

In [31]:
from itertools import product

In [32]:
def partitioning(individuo_a, individuo_b):
    
    subtree_a = individuo_a.chromosome
    subtree_b = individuo_b.chromosome

    partition = Graph()
    offspring = Graph()
    
    for v, w in subtree_a.gen_undirect_edges():
        weight = subtree_a.weight(v,w)

        if subtree_b.has_edge(v, w):
            offspring.add_edge(v, w, weight=weight)
        else:
            partition.add_edge(v, w, weight=weight)
            
    for v, w in subtree_b.gen_undirect_edges():
        weight = subtree_b.weight(v,w)

        if not subtree_a.has_edge(v, w):
            partition.add_edge(v, w, weight=weight)
            
    return partition, offspring 

In [33]:
individuos = [ x for x in populacao ]

In [34]:
for individuo in individuos:
    custo = individuo.cost
    print(custo, end='\t')

265	311	265	319	277	258	285	269	287	318	298	279	279	240	283	297	263	310	251	268	289	332	320	342	276	292	247	284	304	272	244	317	287	232	290	240	300	301	278	248	251	282	249	232	308	244	322	277	250	316	

In [35]:
prod = product(individuos, repeat=2)

counter = 0
counter_cycle = 0 
not_cycle = 0

for a, b in prod:
    p, _ = partitioning(a, b)
    counter += 1
    if has_cycle(p):
        counter_cycle += 1
    else:
        not_cycle += 1

In [36]:
counter

2500

In [37]:
counter_cycle

2446

In [38]:
counter_cycle / counter

0.9784